# 🌐 Hindi Literary Translation Generator for Google Colab

**Enhanced Multilingual Translation with Real-time Streaming**

This notebook allows you to:
1. Upload your text file to translate
2. Select AI provider (HuggingFace or Ollama) and model
3. Choose target language and translation quality tier
4. Generate translation and download the result

**Supported Providers:**
- 🤗 HuggingFace - Various translation models (faster on GPU)
- 🦙 Ollama - Run LLMs locally in Colab (supports many models)

**Recommended Models for Hindi:**
- HuggingFace: `facebook/nllb-200-distilled-600M` - Fast, multilingual
- HuggingFace: `ai4bharat/indictrans2-en-indic-1B` - Best for English→Hindi
- Ollama: `qwen2.5:3b` - Fast, good quality translations
- Ollama: `qwen2.5:7b` - Better quality, balanced speed

## 📦 Step 1: Install Dependencies
Run this cell to install all required packages.

In [ ]:
# Install required packages
!pip install -q torch transformers accelerate sentencepiece
!pip install -q colorama huggingface-hub

# Install Ollama Python client
!pip install -q ollama

# Setup HuggingFace login for gated models
print("\n🔐 HuggingFace Login (for gated models like TranslateGemma):")
print("   If you need access to gated models, run the next cell to login.")
print("   Otherwise, skip to Step 2.\n")
print("✅ All dependencies installed!")


🔐 HuggingFace Login (for gated models like TranslateGemma):
   If you need access to gated models, run the next cell to login.
   Otherwise, skip to Step 2.

✅ All dependencies installed!


### 🔐 (Optional) HuggingFace Login
Run this cell to login to HuggingFace for accessing gated models like `google/translategemma-27b-it`.

In [ ]:
# HuggingFace Login for Gated Models
from huggingface_hub import login, HfFolder
import os

print("🔐 HuggingFace Login Options:")
print("   1. Interactive Login - Opens a browser/token prompt")
print("   2. Token Login - Paste your HF token directly\n")

# Check if already logged in
existing_token = HfFolder.get_token()
if existing_token:
    print(f"✅ Already logged in to HuggingFace!")
    print(f"   Token: {existing_token[:10]}...{existing_token[-5:]}")
else:
    print("📝 Not logged in. Choose a login method:")
    print("\n🔹 Option 1: Interactive Login (recommended)")
    print("   Uncomment the line below and run this cell:")
    print("   # login()\n")
    print("🔹 Option 2: Token Login")
    print("   Get your token from: https://huggingface.co/settings/tokens")
    print("   Then paste it in the HF Token field in Step 3\n")

# Uncomment ONE of the following lines to login:

# Option 1: Interactive login (will prompt for token)
# login()

# Option 2: Direct token login (paste your token)
# login(token="hf_YOUR_TOKEN_HERE")

### 🦙 (Optional) Ollama Setup
Run these cells if you want to use Ollama models. Skip if using HuggingFace only.

In [ ]:
# Install and start Ollama server (required for Ollama models)
import subprocess
import time
import os

print("🦙 Installing Ollama...")

# Install zstd first (required for Ollama extraction)
!apt-get update -qq && apt-get install -y -qq zstd > /dev/null 2>&1

# Download and install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

print("\n🚀 Starting Ollama server in background...")

# Start Ollama server in background
os.environ['OLLAMA_HOST'] = '127.0.0.1:11434'
subprocess.Popen(['/usr/local/bin/ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Wait for server to start
time.sleep(5)

# Verify server is running
try:
    import ollama
    ollama.list()
    print("✅ Ollama server is running!")
except Exception as e:
    print(f"⚠️ Ollama server may not be ready yet. Error: {e}")
    print("   Please wait a few seconds and try running the next cell.")


🦙 Installing Ollama...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

🚀 Starting Ollama server in background...
✅ Ollama server is running!


In [ ]:
# Pull Ollama model (run this cell to download your chosen model)
import ipywidgets as widgets
from IPython.display import display, HTML

print("🦙 Ollama Model Download")
print("=" * 50)

# Model selection for pulling
OLLAMA_MODELS_TO_PULL = {
    "qwen2.5:3b (Fast, ~2GB)": "qwen2.5:3b",
    "qwen2.5:7b (Balanced, ~4GB)": "qwen2.5:7b",
    "deepseek-r1:7b (Reasoning, ~4GB)": "deepseek-r1:7b",
    "llama3.2:3b (Fast, ~2GB)": "llama3.2:3b",
    "mistral:7b (Quality, ~4GB)": "mistral:7b",
    "gemma2:2b (Compact, ~1.5GB)": "gemma2:2b",
    "phi3:mini (Compact, ~2GB)": "phi3:mini"
}

model_pull_dropdown = widgets.Dropdown(
    options=list(OLLAMA_MODELS_TO_PULL.keys()),
    value="qwen2.5:3b (Fast, ~2GB)",
    description='Model to Pull:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

custom_ollama_pull = widgets.Text(
    value='',
    placeholder='Or enter custom model name (e.g., llama3:8b)',
    description='Custom Model:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

display(model_pull_dropdown)
display(custom_ollama_pull)
print("\n💡 Select a model and run the next cell to download it.")

🦙 Ollama Model Download


Dropdown(description='Model to Pull:', layout=Layout(width='400px'), options=('qwen2.5:3b (Fast, ~2GB)', 'qwen…

Text(value='', description='Custom Model:', layout=Layout(width='400px'), placeholder='Or enter custom model n…


💡 Select a model and run the next cell to download it.


In [ ]:
# Actually pull the selected model
import ollama

# Get model to pull
if custom_ollama_pull.value.strip():
    model_to_pull = custom_ollama_pull.value.strip()
else:
    model_to_pull = OLLAMA_MODELS_TO_PULL[model_pull_dropdown.value]

print(f"📥 Pulling model: {model_to_pull}")
print("   This may take several minutes depending on model size...\n")

try:
    # Pull with progress
    current_digest = ''
    for progress in ollama.pull(model_to_pull, stream=True):
        digest = progress.get('digest', '')
        if digest != current_digest and current_digest:
            print()  # Newline between layers
        current_digest = digest

        status = progress.get('status', '')
        if 'completed' in progress and 'total' in progress:
            completed = progress['completed']
            total = progress['total']
            pct = (completed / total * 100) if total > 0 else 0
            print(f"\r   {status}: {pct:.1f}% ({completed}/{total})", end='', flush=True)
        else:
            print(f"\r   {status}", end='', flush=True)

    print(f"\n\n✅ Model '{model_to_pull}' pulled successfully!")

    # List available models
    print("\n📋 Available Ollama models:")
    models = ollama.list()
    for model in models.get('models', []):
        name = model.get('name', 'unknown')
        size = model.get('size', 0) / (1024**3)  # Convert to GB
        print(f"   • {name} ({size:.2f} GB)")

except Exception as e:
    print(f"\n❌ Error pulling model: {e}")
    print("   Make sure Ollama server is running (run the previous cell first).")

📥 Pulling model: translategemma:27b
   This may take several minutes depending on model size...

   pulling d87ec9a33af4: 100.0% (17358640928/17358640928)
   pulling e0a42594d802: 100.0% (358/358)
   pulling 3e2c24001f9e: 100.0% (8431/8431)
   pulling 339e884a40f6: 100.0% (61/61)
   pulling 80f9232a32b9: 100.0% (490/490)
   success

✅ Model 'translategemma:27b' pulled successfully!

📋 Available Ollama models:
   • unknown (16.17 GB)
   • unknown (3.07 GB)


## 📤 Step 2: Upload Your Text File
Upload the text file you want to translate.

In [19]:
from google.colab import files
import os

print("📤 Please upload your text file to translate:")
uploaded = files.upload()

# Get the uploaded file name
UPLOADED_FILE = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {UPLOADED_FILE}")
print(f"📄 File size: {len(uploaded[UPLOADED_FILE])} bytes")

# Display preview
with open(UPLOADED_FILE, 'r', encoding='utf-8') as f:
    content = f.read()
    word_count = len(content.split())
    char_count = len(content)

print(f"\n📊 Content stats:")
print(f"   Words: {word_count:,}")
print(f"   Characters: {char_count:,}")
print(f"\n📝 Preview (first 500 chars):\n{content[:500]}...")

📤 Please upload your text file to translate:


Saving CHAPTER 1.txt to CHAPTER 1 (1).txt

✅ Uploaded: CHAPTER 1 (1).txt
📄 File size: 17212 bytes

📊 Content stats:
   Words: 2,949
   Characters: 17,184

📝 Preview (first 500 chars):
CHAPTER 1 
I am by birth a Genevese; and my family is one of the most distinguished of that republic. My ancestors had been for
many years counsellors and syndics; and my father had filled several public situations with honour and reputation. He
was respected by all who knew him for his integrity and indefatigable attention to public business. He passed his
younger days perpetually occupied by the affairs of his country; and it was not until the decline of life that he thought of
marrying, and b...


## ⚙️ Step 3: Select AI Provider, Model & Language
Choose your preferred translation model and settings.

In [20]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Provider options
PROVIDER_OPTIONS = {
    "HuggingFace (Recommended for Colab)": "huggingface",
    "Ollama (Local only)": "ollama"
}

# Model options by provider
HF_MODEL_OPTIONS = {
    "facebook/nllb-200-distilled-600M (Fast, Multilingual)": "facebook/nllb-200-distilled-600M",
    "facebook/nllb-200-1.3B (Better Quality)": "facebook/nllb-200-1.3B",
    "ai4bharat/indictrans2-en-indic-1B (Best English→Hindi)": "ai4bharat/indictrans2-en-indic-1B",
    "google/madlad400-3b-mt (High Quality, Slow)": "google/madlad400-3b-mt",
    "Helsinki-NLP/opus-mt-en-hi (Simple EN→HI)": "Helsinki-NLP/opus-mt-en-hi",
    "tencent/HY-MT1.5-7B (Hunyuan MT - Best Quality)": "tencent/HY-MT1.5-7B",
    "tencent/HY-MT1.5-1.8B (Hunyuan MT - Fast)": "tencent/HY-MT1.5-1.8B",
    "google/translategemma-27b-it (🔐 Gated - Requires Login)": "google/translategemma-27b-it",
    "Custom Model (enter below)": "custom"
}

OLLAMA_MODEL_OPTIONS = {
    "qwen2.5:3b (Fast)": "qwen2.5:3b",
    "qwen2.5:7b (Balanced)": "qwen2.5:7b",
    "deepseek-r1:7b (Reasoning)": "deepseek-r1:7b",
    "llama3.2:3b (Fast)": "llama3.2:3b",
    "Custom Model (enter below)": "custom"
}

# Language options (NLLB language codes)
LANGUAGE_OPTIONS = {
    "Hindi (हिन्दी)": "hin_Deva",
    "Bengali (বাংলা)": "ben_Beng",
    "Tamil (தமிழ்)": "tam_Taml",
    "Telugu (తెలుగు)": "tel_Telu",
    "Marathi (मराठी)": "mar_Deva",
    "Gujarati (ગુજરાતી)": "guj_Gujr",
    "Kannada (ಕನ್ನಡ)": "kan_Knda",
    "Malayalam (മലയാളം)": "mal_Mlym",
    "Punjabi (ਪੰਜਾਬੀ)": "pan_Guru",
    "Odia (ଓଡ଼ିଆ)": "ory_Orya",
    "Urdu (اردو)": "urd_Arab",
    "Spanish (Español)": "spa_Latn",
    "French (Français)": "fra_Latn",
    "German (Deutsch)": "deu_Latn",
    "Chinese (中文)": "zho_Hans",
    "Japanese (日本語)": "jpn_Jpan",
    "Custom (enter code below)": "custom"
}

# Translation quality tiers
TIER_OPTIONS = {
    "BASIC - Fast, good quality": "BASIC",
    "INTERMEDIATE - Balanced (recommended)": "INTERMEDIATE",
    "ADVANCED - Best quality, slower": "ADVANCED"
}

# Provider dropdown
provider_dropdown = widgets.Dropdown(
    options=list(PROVIDER_OPTIONS.keys()),
    value="HuggingFace (Recommended for Colab)",
    description='Provider:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Model dropdown (HuggingFace by default)
model_dropdown = widgets.Dropdown(
    options=list(HF_MODEL_OPTIONS.keys()),
    value="facebook/nllb-200-distilled-600M (Fast, Multilingual)",
    description='Model:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Custom model input
custom_model_input = widgets.Text(
    value='',
    placeholder='Enter HuggingFace model name',
    description='Custom Model:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Target language dropdown
language_dropdown = widgets.Dropdown(
    options=list(LANGUAGE_OPTIONS.keys()),
    value="Hindi (हिन्दी)",
    description='Target Language:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Custom language code input
custom_language_input = widgets.Text(
    value='',
    placeholder='Enter NLLB language code (e.g., hin_Deva)',
    description='Custom Lang:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Translation tier dropdown
tier_dropdown = widgets.Dropdown(
    options=list(TIER_OPTIONS.keys()),
    value="INTERMEDIATE - Balanced (recommended)",
    description='Quality:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# HuggingFace token (optional)
hf_token_input = widgets.Password(
    value='',
    placeholder='Optional: HF token for gated models',
    description='HF Token:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Chunk size slider
chunk_size_slider = widgets.IntSlider(
    value=350,
    min=100,
    max=1000,
    step=50,
    description='Chunk Size:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

display(HTML("<h3>🎛️ Configure Translation Settings</h3>"))
display(provider_dropdown)
display(model_dropdown)
display(custom_model_input)
display(HTML("<br>"))
display(language_dropdown)
display(custom_language_input)
display(HTML("<br>"))
display(tier_dropdown)
display(chunk_size_slider)
display(hf_token_input)


# Handle provider change to update model dropdown
def on_provider_change(change):
    if change['new'] == "HuggingFace (Recommended for Colab)":
        model_dropdown.options = list(HF_MODEL_OPTIONS.keys())
        model_dropdown.value = "facebook/nllb-200-distilled-600M (Fast, Multilingual)"
        custom_model_input.placeholder = 'Enter HuggingFace model name'
    else:
        model_dropdown.options = list(OLLAMA_MODEL_OPTIONS.keys())
        model_dropdown.value = "qwen2.5:3b (Fast)"
        custom_model_input.placeholder = 'Enter Ollama model name'

provider_dropdown.observe(on_provider_change, names='value')
print("\n💡 Tip: facebook/nllb-200-distilled-600M is recommended for fast Hindi translation!")
print("🔐 Note: Models marked with 🔐 require HuggingFace login. Run the login cell above first.")

Dropdown(description='Provider:', layout=Layout(width='400px'), options=('HuggingFace (Recommended for Colab)'…

Dropdown(description='Model:', layout=Layout(width='500px'), options=('facebook/nllb-200-distilled-600M (Fast,…

Text(value='', description='Custom Model:', layout=Layout(width='500px'), placeholder='Enter HuggingFace model…

Dropdown(description='Target Language:', layout=Layout(width='400px'), options=('Hindi (हिन्दी)', 'Bengali (বা…

Text(value='', description='Custom Lang:', layout=Layout(width='400px'), placeholder='Enter NLLB language code…

Dropdown(description='Quality:', index=1, layout=Layout(width='400px'), options=('BASIC - Fast, good quality',…

IntSlider(value=350, description='Chunk Size:', layout=Layout(width='400px'), max=1000, min=100, step=50, styl…

Password(description='HF Token:', layout=Layout(width='400px'), placeholder='Optional: HF token for gated mode…


💡 Tip: facebook/nllb-200-distilled-600M is recommended for fast Hindi translation!
🔐 Note: Models marked with 🔐 require HuggingFace login. Run the login cell above first.


In [21]:
# Store the selected configuration
SELECTED_PROVIDER = PROVIDER_OPTIONS[provider_dropdown.value]

# Get model based on provider
if SELECTED_PROVIDER == "huggingface":
    selected_model_key = model_dropdown.value
    SELECTED_MODEL = HF_MODEL_OPTIONS.get(selected_model_key, "custom")
else:
    SELECTED_MODEL = OLLAMA_MODEL_OPTIONS.get(model_dropdown.value, "custom")

if SELECTED_MODEL == "custom":
    SELECTED_MODEL = custom_model_input.value
    if not SELECTED_MODEL:
        raise ValueError("Please enter a custom model name!")

# Get target language
TARGET_LANGUAGE = LANGUAGE_OPTIONS[language_dropdown.value]
if TARGET_LANGUAGE == "custom":
    TARGET_LANGUAGE = custom_language_input.value
    if not TARGET_LANGUAGE:
        raise ValueError("Please enter a custom language code!")

TRANSLATION_TIER = TIER_OPTIONS[tier_dropdown.value]
CHUNK_SIZE = chunk_size_slider.value
HF_TOKEN = hf_token_input.value if hf_token_input.value else None

print(f"\n✅ Configuration saved:")
print(f"   🤖 Provider: {SELECTED_PROVIDER}")
print(f"   📦 Model: {SELECTED_MODEL}")
print(f"   🌐 Target Language: {TARGET_LANGUAGE}")
print(f"   🎯 Quality Tier: {TRANSLATION_TIER}")
print(f"   📦 Chunk Size: {CHUNK_SIZE} words")
print(f"   🔑 HF Token: {'Provided' if HF_TOKEN else 'Not provided'}")


✅ Configuration saved:
   🤖 Provider: ollama
   📦 Model: translategemma:27b
   🌐 Target Language: hin_Deva
   🎯 Quality Tier: ADVANCED
   📦 Chunk Size: 350 words
   🔑 HF Token: Not provided


## 🚀 Step 4: Translation Engine Setup
This cell contains the complete translation engine code.

In [22]:
#!/usr/bin/env python3
"""
Enhanced Translation Engine for Google Colab
Supports HuggingFace models with multiple language targets
"""

import os
import sys
import json
import time
import warnings
import re
from pathlib import Path
from datetime import datetime

warnings.filterwarnings("ignore")

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline

# Check GPU availability
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Using device: {DEVICE}")
if DEVICE == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


# Translation Prompts for LLM-based models
TRANSLATION_PROMPTS = {
    "BASIC": {
        "system": """You are a professional translator. Translate the text accurately.""",
        "user": """Translate the following text to {target_lang}:\n\n{chunk}\n\nTranslation:"""
    },
    "INTERMEDIATE": {
        "system": """You are an expert literary translator. Create translations that feel natural in the target language while preserving all meaning and nuance.""",
        "user": """Translate the following text to {target_lang}. Maintain all details, dialogue, and descriptions:\n\n{chunk}\n\nComplete Translation:"""
    },
    "ADVANCED": {
        "system": """You are a master literary translator. Your translations should feel like they were originally written in the target language by a native speaker. Preserve every sentence, every detail, every nuance.\nRequirements:\n- Translate EVERY sentence\n- Maintain ALL dialogue\n- Preserve ALL descriptions\n- Keep similar length\n\n""",
        "user": """Translate the COMPLETE passage below to {target_lang}.\n\nText:\n\n{chunk}"""
    }
}

# Language name mappings
LANG_NAMES = {
    'hin_Deva': 'Hindi', 'ben_Beng': 'Bengali', 'tam_Taml': 'Tamil',
    'tel_Telu': 'Telugu', 'mar_Deva': 'Marathi', 'guj_Gujr': 'Gujarati',
    'kan_Knda': 'Kannada', 'mal_Mlym': 'Malayalam', 'pan_Guru': 'Punjabi',
    'ory_Orya': 'Odia', 'urd_Arab': 'Urdu', 'spa_Latn': 'Spanish',
    'fra_Latn': 'French', 'deu_Latn': 'German', 'zho_Hans': 'Chinese',
    'jpn_Jpan': 'Japanese'
}


def chunk_text(text, chunk_words=350):
    """Split text into chunks at paragraph boundaries."""
    paragraph_patterns = [
        r'\n\s*\n',
        r'\r\n\s*\r\n',
        r'\n\s{2,}\n',
    ]
    paragraph_split_pattern = '|'.join(paragraph_patterns)
    paragraphs = re.split(paragraph_split_pattern, text)
    paragraphs = [para.strip() for para in paragraphs if para.strip()]

    chunks = []
    current_chunk = []
    current_count = 0

    for para in paragraphs:
        para_words = para.split()
        para_count = len(para_words)

        if para_count > chunk_words:
            if current_chunk:
                chunks.append('\n\n'.join(current_chunk))
                current_chunk = []
                current_count = 0

            words = para.split()
            for i in range(0, len(words), chunk_words):
                chunk_words_list = words[i:i + chunk_words]
                chunk_text = ' '.join(chunk_words_list)
                chunks.append(chunk_text)
        else:
            if current_count + para_count > chunk_words and current_chunk:
                chunks.append('\n\n'.join(current_chunk))
                current_chunk = [para]
                current_count = para_count
            else:
                current_chunk.append(para)
                current_count += para_count

    if current_chunk:
        chunks.append('\n\n'.join(current_chunk))

    return chunks


def clean_translation(text):
    """Clean up translation artifacts."""
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    text = re.sub(r'```\w*\n?', '', text)
    text = re.sub(r'(Translation:|Hindi Translation:|Here\'s the translation:)', '', text, flags=re.IGNORECASE)
    lines = [line.strip() for line in text.split('\n')]
    text = '\n\n'.join(line for line in lines if line)
    return text.strip()


class TranslationEngine:
    """Translation engine with HuggingFace support."""

    def __init__(self, model_name, target_lang, device="cuda", hf_token=None):
        self.model_name = model_name
        self.target_lang = target_lang
        self.device = device
        self.hf_token = hf_token
        self.model = None
        self.tokenizer = None
        self.translator = None
        self.model_type = self._detect_model_type(model_name)

        # Try to get token from HuggingFace Hub if not provided
        if not self.hf_token:
            try:
                from huggingface_hub import HfFolder
                self.hf_token = HfFolder.get_token()
            except:
                pass

        if self.hf_token:
            os.environ['HF_TOKEN'] = self.hf_token

        self.load_model()

    def _detect_model_type(self, model_name):
        """Detect model type from name."""
        model_lower = model_name.lower()
        if 'nllb' in model_lower:
            return 'nllb'
        elif 'indictrans' in model_lower:
            return 'indictrans'
        elif 'opus-mt' in model_lower or 'helsinki' in model_lower:
            return 'opus'
        elif 'madlad' in model_lower:
            return 'madlad'
        elif 'mbart' in model_lower:
            return 'mbart'
        elif 'hy-mt' in model_lower or 'hunyuan' in model_lower:
            return 'hymt'  # Tencent Hunyuan MT
        elif 'translategemma' in model_lower:
            return 'translategemma'  # Google TranslateGemma
        else:
            return 'causal'  # LLM-based translation

    def load_model(self):
        """Load translation model."""
        print(f"📥 Loading model: {self.model_name}")
        print(f"   Model type: {self.model_type}")

        try:
            if self.model_type in ['nllb', 'opus', 'mbart']:
                self._load_seq2seq_model()
            elif self.model_type == 'indictrans':
                self._load_indictrans_model()
            elif self.model_type == 'madlad':
                self._load_madlad_model()
            elif self.model_type == 'hymt':
                self._load_hymt_model()
            elif self.model_type == 'translategemma':
                self._load_translategemma_model()
            else:
                self._load_causal_model()

            print("✅ Model loaded successfully!")
        except Exception as e:
            print(f"❌ Failed to load model: {e}")
            raise

    def _load_seq2seq_model(self):
        """Load Seq2Seq translation model (NLLB, OPUS, mBART)."""
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            token=self.hf_token,
            src_lang="eng_Latn" if self.model_type == 'nllb' else None
        )
        self.model = AutoModelForSeq2SeqLM.from_pretrained(
            self.model_name,
            token=self.hf_token,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto" if self.device == "cuda" else None
        )

        if self.device != "cuda":
            self.model = self.model.to(self.device)

        self.translator = pipeline(
            "translation",
            model=self.model,
            tokenizer=self.tokenizer,
            src_lang="eng_Latn" if self.model_type == 'nllb' else "en",
            tgt_lang=self.target_lang if self.model_type == 'nllb' else None,
            max_length=1024,
            device=0 if self.device == "cuda" else -1
        )

    def _load_indictrans_model(self):
        """Load IndicTrans2 model."""
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            token=self.hf_token,
            trust_remote_code=True
        )
        self.model = AutoModelForSeq2SeqLM.from_pretrained(
            self.model_name,
            token=self.hf_token,
            trust_remote_code=True,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
        )

        if self.device == "cuda":
            self.model = self.model.cuda()

    def _load_madlad_model(self):
        """Load MADLAD-400 model."""
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            token=self.hf_token
        )
        self.model = AutoModelForSeq2SeqLM.from_pretrained(
            self.model_name,
            token=self.hf_token,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto" if self.device == "cuda" else None
        )

    def _load_hymt_model(self):
        """Load Tencent Hunyuan MT model."""
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            token=self.hf_token,
            trust_remote_code=True
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            token=self.hf_token,
            trust_remote_code=True,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto" if self.device == "cuda" else None
        )

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def _load_translategemma_model(self):
        """Load Google TranslateGemma model (gated)."""
        # Check for token
        if not self.hf_token:
            raise ValueError("TranslateGemma is a gated model. Please login to HuggingFace first (run the login cell) or provide HF Token.")

        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            token=self.hf_token
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            token=self.hf_token,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto" if self.device == "cuda" else None
        )

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def _load_causal_model(self):
        """Load causal LM for translation."""
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            token=self.hf_token
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            token=self.hf_token,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto" if self.device == "cuda" else None
        )

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def _filter_inputs(self, inputs):
        """Filter out unsupported model kwargs like token_type_ids."""
        # Remove token_type_ids if present (many models don't use them)
        if 'token_type_ids' in inputs:
            del inputs['token_type_ids']
        return inputs

    def translate(self, text, tier="INTERMEDIATE"):
        """Translate text based on model type."""
        if self.model_type in ['nllb', 'opus', 'mbart']:
            return self._translate_seq2seq(text)
        elif self.model_type == 'indictrans':
            return self._translate_indictrans(text)
        elif self.model_type == 'madlad':
            return self._translate_madlad(text)
        elif self.model_type == 'hymt':
            return self._translate_hymt(text)
        elif self.model_type == 'translategemma':
            return self._translate_translategemma(text)
        else:
            return self._translate_causal(text, tier)

    def _translate_seq2seq(self, text):
        """Translate using Seq2Seq model."""
        result = self.translator(text, max_length=1024)
        return result[0]['translation_text']

    def _translate_indictrans(self, text):
        """Translate using IndicTrans2."""
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = self._filter_inputs(inputs)

        if self.device == "cuda":
            inputs = {k: v.cuda() for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=512,
                num_beams=5,
                num_return_sequences=1
            )

        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def _translate_madlad(self, text):
        """Translate using MADLAD-400."""
        # MADLAD uses language tags like <2hi> for Hindi
        lang_code = self.target_lang.split('_')[0][:2]  # Extract 2-letter code
        tagged_text = f"<2{lang_code}> {text}"

        inputs = self.tokenizer(tagged_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = self._filter_inputs(inputs)

        if self.device == "cuda":
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=512,
                num_beams=4
            )

        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def _translate_hymt(self, text):
        """Translate using Tencent Hunyuan MT."""
        lang_name = LANG_NAMES.get(self.target_lang, self.target_lang)

        # Hunyuan MT uses a simple prompt format
        prompt = f"Translate the following text to {lang_name}:\n{text}\n\nTranslation:"

        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048)
        inputs = self._filter_inputs(inputs)  # Remove token_type_ids

        if self.device == "cuda":
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=1024,
                temperature=0.3,
                do_sample=True,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )

        generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract translation after the prompt
        if 'Translation:' in generated:
            translation = generated.split('Translation:')[-1].strip()
        else:
            translation = generated[len(prompt):].strip()

        return translation

    def _translate_translategemma(self, text):
        """Translate using Google TranslateGemma."""
        lang_name = LANG_NAMES.get(self.target_lang, self.target_lang)

        # TranslateGemma instruction format
        prompt = f"Translate the following text from English to {lang_name}:\n\n{text}\n\nTranslation:"

        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048)
        inputs = self._filter_inputs(inputs)  # Remove token_type_ids

        if self.device == "cuda":
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=1024,
                temperature=0.3,
                do_sample=True,
                pad_token_id=self.tokenizer.pad_token_id
            )

        generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract translation after the prompt
        if 'Translation:' in generated:
            translation = generated.split('Translation:')[-1].strip()
        else:
            translation = generated[len(prompt):].strip()

        return translation

    def _translate_causal(self, text, tier):
        """Translate using causal LM with prompts."""
        prompts = TRANSLATION_PROMPTS[tier]

        # Get language name from code
        lang_name = LANG_NAMES.get(self.target_lang, self.target_lang)

        full_prompt = f"{prompts['system']}\n\n{prompts['user'].format(target_lang=lang_name, chunk=text)}"

        inputs = self.tokenizer(full_prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048)
        inputs = self._filter_inputs(inputs)  # Remove token_type_ids

        if self.device == "cuda":
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=1024,
                temperature=0.4,
                do_sample=True,
                pad_token_id=self.tokenizer.pad_token_id
            )

        generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract translation from response
        if "Translation:" in generated:
            return generated.split("Translation:")[-1].strip()
        return generated[len(full_prompt):].strip()


class TranslationGenerator:
    """Main translation generator class."""

    def __init__(self, model_name, target_lang, device="cuda", output_dir=".", tier="INTERMEDIATE", chunk_size=350, hf_token=None):
        self.model_name = model_name
        self.target_lang = target_lang
        self.device = device
        self.output_dir = Path(output_dir)
        self.tier = tier
        self.chunk_size = chunk_size

        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.engine = TranslationEngine(model_name, target_lang, device, hf_token)

    def translate_file(self, input_file):
        """Translate entire file."""
        print(f"\n{'=' * 70}")
        print(f"🌐 TRANSLATION GENERATOR")
        print(f"{'=' * 70}")
        print(f"📄 Input: {input_file}")
        print(f"🤖 Model: {self.model_name}")
        print(f"🌐 Target: {self.target_lang}")
        print(f"🎯 Quality: {self.tier}")
        print(f"🖥️ Device: {self.device}")
        print(f"{'=' * 70}\n")

        # Read input
        with open(input_file, 'r', encoding='utf-8') as f:
            text = f.read()

        # Clean markers
        lines = text.split('\n')
        cleaned = [l for l in lines if not (l.strip().startswith('===') and l.strip().endswith('==='))]
        text = '\n'.join(cleaned).strip()

        orig_words = len(text.split())
        orig_chars = len(text)
        print(f"📊 Input: {orig_chars:,} chars, {orig_words:,} words")

        # Chunk text
        print(f"\n📦 Creating chunks ({self.chunk_size} words each)...")
        chunks = chunk_text(text, self.chunk_size)
        print(f"✅ Created {len(chunks)} chunks")

        # Translate chunks
        print(f"\n🎯 STARTING TRANSLATION\n")

        translations = []
        start_time = time.time()

        for i, chunk in enumerate(chunks, 1):
            chunk_start = time.time()

            print(f"\n{'=' * 50}")
            print(f"📄 Chunk {i}/{len(chunks)}")
            print(f"   Input: {len(chunk.split())} words, {len(chunk)} chars")

            try:
                translated = self.engine.translate(chunk, self.tier)
                translated = clean_translation(translated)
                translations.append(translated)

                chunk_time = time.time() - chunk_start
                print(f"   Output: {len(translated)} chars")
                print(f"   ✅ Completed in {chunk_time:.1f}s")

                # Progress
                elapsed = time.time() - start_time
                avg = elapsed / i
                remaining = len(chunks) - i
                eta = remaining * avg
                print(f"   📈 Progress: {i/len(chunks)*100:.1f}% | ETA: {eta/60:.1f}m")

            except Exception as e:
                print(f"   ❌ Error: {e}")
                translations.append(f"[TRANSLATION ERROR: {e}]")

        # Combine translations
        final_translation = "\n\n".join(translations)

        # Save output
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        lang_code = self.target_lang.split('_')[0]
        output_file = self.output_dir / f"translation_{lang_code}_{timestamp}.txt"

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(final_translation)

        # Summary
        total_time = time.time() - start_time
        trans_chars = len(final_translation)

        print(f"\n{'=' * 70}")
        print(f"🎉 TRANSLATION COMPLETE!")
        print(f"{'=' * 70}")
        print(f"⏱️ Time: {total_time/60:.1f} minutes")
        print(f"📦 Chunks: {len(chunks)}")
        print(f"⚡ Avg/chunk: {total_time/len(chunks):.1f}s")
        print(f"📝 Input: {orig_chars:,} chars")
        print(f"📝 Output: {trans_chars:,} chars")
        print(f"📊 Ratio: {trans_chars/orig_chars:.2f}x")
        print(f"💾 Output: {output_file}")
        print(f"{'=' * 70}")

        return str(output_file)




# ============= OLLAMA TRANSLATION ENGINE =============

class OllamaTranslationEngine:
    """Translation engine using Ollama local models."""

    def __init__(self, model_name, target_lang, tier="INTERMEDIATE"):
        self.model_name = model_name
        self.target_lang = target_lang
        self.tier = tier
        self.lang_name = LANG_NAMES.get(target_lang, target_lang)

        print(f"📥 Initializing Ollama engine with model: {model_name}")
        print(f"   Target language: {self.lang_name}")

        # Verify model is available
        try:
            import ollama
            self.client = ollama
            models = ollama.list()
            available = [m.get('name', '').split(':')[0] for m in models.get('models', [])]
            model_base = model_name.split(':')[0]

            if not any(model_base in m for m in available):
                print(f"⚠️ Model '{model_name}' not found locally. Attempting to pull...")
                ollama.pull(model_name)
                print(f"✅ Model '{model_name}' pulled successfully!")
            else:
                print(f"✅ Model '{model_name}' is available!")
        except Exception as e:
            print(f"❌ Error initializing Ollama: {e}")
            raise

    def translate(self, text, tier=None):
        """Translate text using Ollama model."""
        if tier:
            self.tier = tier

        prompts = TRANSLATION_PROMPTS[self.tier]

        system_prompt = prompts['system']
        user_prompt = prompts['user'].format(target_lang=self.lang_name, chunk=text)

        try:
            response = self.client.chat(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                options={
                    "temperature": 0.3,
                    "num_predict": 2048,
                }
            )

            translation = response['message']['content']
            return clean_translation(translation)

        except Exception as e:
            print(f"❌ Ollama translation error: {e}")
            raise


class OllamaTranslationGenerator:
    """Translation generator using Ollama models."""

    def __init__(self, model_name, target_lang, output_dir=".", tier="INTERMEDIATE", chunk_size=350):
        self.model_name = model_name
        self.target_lang = target_lang
        self.output_dir = Path(output_dir)
        self.tier = tier
        self.chunk_size = chunk_size

        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.engine = OllamaTranslationEngine(model_name, target_lang, tier)

    def translate_file(self, input_file):
        """Translate entire file using Ollama."""
        print(f"\n{'=' * 70}")
        print(f"🌐 OLLAMA TRANSLATION GENERATOR")
        print(f"{'=' * 70}")
        print(f"📄 Input: {input_file}")
        print(f"🦙 Model: {self.model_name}")
        print(f"🌐 Target: {self.target_lang}")
        print(f"🎯 Quality: {self.tier}")
        print(f"{'=' * 70}\n")

        # Read input
        with open(input_file, 'r', encoding='utf-8') as f:
            text = f.read()

        # Clean markers
        lines = text.split('\n')
        cleaned = [l for l in lines if not (l.strip().startswith('===') and l.strip().endswith('==='))]
        text = '\n'.join(cleaned).strip()

        orig_words = len(text.split())
        orig_chars = len(text)
        print(f"📊 Input: {orig_chars:,} chars, {orig_words:,} words")

        # Chunk text
        print(f"\n📦 Creating chunks ({self.chunk_size} words each)...")
        chunks = chunk_text(text, self.chunk_size)
        print(f"✅ Created {len(chunks)} chunks")

        # Translate chunks
        print(f"\n🎯 STARTING TRANSLATION\n")

        translations = []
        start_time = time.time()

        for i, chunk in enumerate(chunks, 1):
            chunk_start = time.time()

            print(f"\n{'=' * 50}")
            print(f"📄 Chunk {i}/{len(chunks)}")
            print(f"   Input: {len(chunk.split())} words, {len(chunk)} chars")

            try:
                translated = self.engine.translate(chunk, self.tier)
                translations.append(translated)

                chunk_time = time.time() - chunk_start
                print(f"   Output: {len(translated)} chars")
                print(f"   ✅ Completed in {chunk_time:.1f}s")

                # Progress
                elapsed = time.time() - start_time
                avg = elapsed / i
                remaining = len(chunks) - i
                eta = remaining * avg
                print(f"   📈 Progress: {i/len(chunks)*100:.1f}% | ETA: {eta/60:.1f}m")

            except Exception as e:
                print(f"   ❌ Error: {e}")
                translations.append(f"[TRANSLATION ERROR: {e}]")

        # Combine translations
        final_translation = "\n\n".join(translations)

        # Save output
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        lang_code = self.target_lang.split('_')[0]
        output_file = self.output_dir / f"translation_{lang_code}_{timestamp}.txt"

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(final_translation)

        # Summary
        total_time = time.time() - start_time
        trans_chars = len(final_translation)

        print(f"\n{'=' * 70}")
        print(f"🎉 TRANSLATION COMPLETE!")
        print(f"{'=' * 70}")
        print(f"⏱️ Time: {total_time/60:.1f} minutes")
        print(f"📦 Chunks: {len(chunks)}")
        print(f"⚡ Avg/chunk: {total_time/len(chunks):.1f}s")
        print(f"📝 Input: {orig_chars:,} chars")
        print(f"📝 Output: {trans_chars:,} chars")
        print(f"📊 Ratio: {trans_chars/orig_chars:.2f}x")
        print(f"💾 Output: {output_file}")
        print(f"{'=' * 70}")

        return str(output_file)


print("✅ Translation Engine loaded and ready!")

🖥️ Using device: cuda
   GPU: Tesla T4
   Memory: 15.83 GB
✅ Translation Engine loaded and ready!


## 🌐 Step 5: Generate Translation
Run this cell to translate your uploaded file.

In [23]:
# Create output directory
OUTPUT_DIR = "./translation_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Initialize the generator based on provider
print("🚀 Initializing Translation Generator...")
print(f"   Provider: {SELECTED_PROVIDER}")
print(f"   Model: {SELECTED_MODEL}")

if SELECTED_PROVIDER == "ollama":
    # Use Ollama generator
    generator = OllamaTranslationGenerator(
        model_name=SELECTED_MODEL,
        target_lang=TARGET_LANGUAGE,
        output_dir=OUTPUT_DIR,
        tier=TRANSLATION_TIER,
        chunk_size=CHUNK_SIZE
    )
else:
    # Use HuggingFace generator
    generator = TranslationGenerator(
        model_name=SELECTED_MODEL,
        target_lang=TARGET_LANGUAGE,
        device=DEVICE,
        output_dir=OUTPUT_DIR,
        tier=TRANSLATION_TIER,
        chunk_size=CHUNK_SIZE,
        hf_token=HF_TOKEN
    )

# Translate
print(f"\n🌐 Starting translation...")
OUTPUT_FILE = generator.translate_file(UPLOADED_FILE)

print(f"\n✅ Translation file generated: {OUTPUT_FILE}")

🚀 Initializing Translation Generator...
   Provider: ollama
   Model: translategemma:27b
📥 Initializing Ollama engine with model: translategemma:27b
   Target language: Hindi
⚠️ Model 'translategemma:27b' not found locally. Attempting to pull...
✅ Model 'translategemma:27b' pulled successfully!

🌐 Starting translation...

🌐 OLLAMA TRANSLATION GENERATOR
📄 Input: CHAPTER 1 (1).txt
🦙 Model: translategemma:27b
🌐 Target: hin_Deva
🎯 Quality: ADVANCED

📊 Input: 17,181 chars, 2,949 words

📦 Creating chunks (350 words each)...
✅ Created 9 chunks

🎯 STARTING TRANSLATION


📄 Chunk 1/9
   Input: 350 words, 2067 chars
   Output: 2009 chars
   ✅ Completed in 285.9s
   📈 Progress: 11.1% | ETA: 38.1m

📄 Chunk 2/9
   Input: 350 words, 1988 chars
   Output: 1958 chars
   ✅ Completed in 219.6s
   📈 Progress: 22.2% | ETA: 29.5m

📄 Chunk 3/9
   Input: 350 words, 2023 chars
   Output: 1982 chars
   ✅ Completed in 226.7s
   📈 Progress: 33.3% | ETA: 24.4m

📄 Chunk 4/9
   Input: 350 words, 1984 chars
   Output

## 📖 Step 6: Preview & Download Translation
View your translation and download it.

In [ ]:
from IPython.display import display, HTML
import os

if os.path.exists(OUTPUT_FILE):
    # Read and display translation
    with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
        translation = f.read()

    file_size = os.path.getsize(OUTPUT_FILE) / 1024  # KB
    word_count = len(translation.split())

    print(f"📊 Translation stats:")
    print(f"   Words: {word_count:,}")
    print(f"   Characters: {len(translation):,}")
    print(f"   File size: {file_size:.2f} KB")

    print(f"\n📖 Preview (first 1000 chars):")
    print(f"{'=' * 50}")
    print(translation[:1000])
    print(f"{'=' * 50}")
    if len(translation) > 1000:
        print(f"... [truncated, {len(translation) - 1000:,} more chars]")
else:
    print("❌ Output file not found. Please run the translation step again.")

In [24]:
# Download the translated file
from google.colab import files

print("📥 Downloading your translated file...")
files.download(OUTPUT_FILE)
print("✅ Download started! Check your browser's download folder.")

📥 Downloading your translated file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download started! Check your browser's download folder.


## 💾 (Optional) Save to Google Drive
If you want to save the translation to your Google Drive.

In [ ]:
# Mount Google Drive
from google.colab import drive
import shutil

print("📂 Mounting Google Drive...")
drive.mount('/content/drive')

# Create output folder in Drive
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/Translation_Output"
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

# Copy file to Drive
drive_output_path = os.path.join(DRIVE_OUTPUT_DIR, os.path.basename(OUTPUT_FILE))
shutil.copy(OUTPUT_FILE, drive_output_path)

print(f"\n✅ Translation saved to Google Drive:")
print(f"   📁 {drive_output_path}")

---

## 📚 Quick Reference

### Supported Languages (NLLB Codes):
| Language | Code |
|----------|------|
| Hindi | `hin_Deva` |
| Bengali | `ben_Beng` |
| Tamil | `tam_Taml` |
| Telugu | `tel_Telu` |
| Marathi | `mar_Deva` |
| Gujarati | `guj_Gujr` |
| Spanish | `spa_Latn` |
| French | `fra_Latn` |
| German | `deu_Latn` |

### Recommended Models:
| Model | Best For | Speed |
|-------|----------|-------|
| `facebook/nllb-200-distilled-600M` | Fast multilingual | ⚡ Fast |
| `facebook/nllb-200-1.3B` | Better quality | 🔄 Medium |
| `ai4bharat/indictrans2-en-indic-1B` | Best EN→Hindi | 🔄 Medium |
| `google/madlad400-3b-mt` | Highest quality | 🐢 Slow |

### Quality Tiers:
- **BASIC**: Fast, good for simple texts
- **INTERMEDIATE**: Balanced quality and speed (recommended)
- **ADVANCED**: Best quality, preserves all nuances

### Tips:
- Use Colab GPU for faster translation
- For long texts, use smaller chunk sizes (200-300 words)
- NLLB models are best for multilingual translation